In [ ]:
import os
import glob
import pandas as pd

In [ ]:
pocket_list = glob.glob("mount/output/*predictions.csv")
os.makedirs("mount/pockets", exist_ok=True)

# Convert P2Rank pockets to PPCalign POC formatted files

In [ ]:
for p2rank_path in pocket_list:
    # Get the path of the 
    pdb_path = os.path.join("mount", "pdbs", os.path.basename(p2rank_path).replace("_predictions.csv", ""))
    # Extract the uniprot ID from the AF2 name
    pdb_id = os.path.basename(p2rank_path).replace(".pdb_predictions.csv", "").split("-")[1]
    
    # Open the P2Rank pocket results
    tmp = pd.read_csv(p2rank_path)
    tmp.columns = [col.replace(" ", "") for col in tmp.columns]
    
    # Filter and keep only high probability pockets
    tmp = tmp[tmp["probability"]>0.9]
    
    # If there are no pockets, skip to the next PDB
    if len(tmp) == 0:
        continue
    
    pocket_dict = {}
    
    # Iterate over every pocket and create a pocket PDB
    for _, row in tmp.iterrows():
        residues = row["residue_ids"][1:].split(" ")
        pocket_name = pdb_id + "_" + row["name"].replace(" ","").replace("pocket", "p")
        pocket_residues = [tuple(res.split("_")) for res in residues]

        with open(pdb_path, 'r') as input_file:
            with open("mount/pockets/{}.pdb".format(pocket_name), 'w') as output_file:
                output_file.write("POC {}\n".format(pocket_name))
                for line in input_file:
                    if line.startswith(('ATOM', 'HETATM')):
                        chain = line[21]
                        residue_number = line[22:26].strip()
                        if (chain, residue_number) in pocket_residues:
                            output_file.write(line)
                output_file.write("TER")